<a href="https://colab.research.google.com/github/bairoy/pytorch/blob/main/imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import utils
from keras.layers import Dense,Activation,Flatten,Dropout,BatchNormalization,Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras import regularizers, optimizers
import numpy as np
from matplotlib import pyplot

In [22]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
(x_train,x_valid) = x_train[5000:],x_train[:5000]
(y_train,y_valid) = y_train[5000:],y_train[:5000]


In [23]:
print(f'x_train = {x_train.shape}')
print(f'x_valid = {x_valid.shape}')
print(f'x_test = {x_test.shape}')


x_train = (45000, 32, 32, 3)
x_valid = (5000, 32, 32, 3)
x_test = (10000, 32, 32, 3)


In [24]:
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_valid = (x_valid-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [25]:
num_classes = 10
y_train = utils.to_categorical(y_train,num_classes)
y_valid = utils.to_categorical(y_valid,num_classes)
y_test = utils.to_categorical(y_test,num_classes)


In [26]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)
datagen.fit(x_train)

In [27]:
base_hidden_units = 32
weight_decay = 1e-4
model = Sequential()

In [28]:
# conv1
model.add(Conv2D(base_hidden_units,kernel_size=3,padding='same',kernel_regularizer=regularizers.l2(weight_decay),input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
# conv2d
model.add(Conv2D(base_hidden_units,kernel_size=3,padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

In [30]:
# pool + dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [31]:
# conv3
model.add(Conv2D(base_hidden_units*2,kernel_size=3,padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

In [32]:
# conv4
model.add(Conv2D(base_hidden_units*2,kernel_size=3,padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())


In [33]:
# pool + dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))


In [34]:
# conv5
model.add(Conv2D(base_hidden_units*4,kernel_size=3,padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

In [35]:
# conv6
model.add(Conv2D(base_hidden_units*4,kernel_size=3,padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())


In [36]:
# pool + dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

In [37]:
# fully connected layer -> 7th layer
model.add(Flatten())
model.add(Dense(10,activation='softmax'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 309,290 (1.18 MB)

 Trainable params: 308,394 (1.18 MB)

 Non-trainable params: 896 (3.50 KB)

In [43]:
batch_size=128
epochs=200
checkpointer = ModelCheckpoint(filepath='model.100epochs.hdf5.keras',verbose=1,save_best_only=True)
optimizer = keras.optimizers.Adam(learning_rate=0.0001,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])


In [ ]:
history = model.fit(datagen.flow(x_train,y_train,batch_size=batch_size),callbacks=[checkpointer],steps_per_epoch=x_train.shape[0]//batch_size,epochs=epochs,verbose=2,validation_data = (x_valid,y_valid))

Epoch 1/200

Epoch 1: val_loss improved from inf to 0.56929, saving model to model.100epochs.hdf5.keras
351/351 - 44s - 125ms/step - accuracy: 0.8069 - loss: 0.5836 - val_accuracy: 0.8226 - val_loss: 0.5693
Epoch 2/200

Epoch 2: val_loss improved from 0.56929 to 0.56759, saving model to model.100epochs.hdf5.keras
351/351 - 1s - 2ms/step - accuracy: 0.8438 - loss: 0.5748 - val_accuracy: 0.8234 - val_loss: 0.5676
Epoch 3/200

Epoch 3: val_loss improved from 0.56759 to 0.54881, saving model to model.100epochs.hdf5.keras
351/351 - 27s - 77ms/step - accuracy: 0.8107 - loss: 0.5763 - val_accuracy: 0.8218 - val_loss: 0.5488
Epoch 4/200

Epoch 4: val_loss improved from 0.54881 to 0.54670, saving model to model.100epochs.hdf5.keras
351/351 - 1s - 2ms/step - accuracy: 0.8281 - loss: 0.6041 - val_accuracy: 0.8216 - val_loss: 0.5467
Epoch 5/200

Epoch 5: val_loss did not improve from 0.54670
351/351 - 26s - 73ms/step - accuracy: 0.8136 - loss: 0.5696 - val_accuracy: 0.8244 - val_loss: 0.5674
Epoch

In [1]:
scores = model.evaluate(x_test,y_test,batch_size=128,verbose=1)
print('\n Test result: %.3f loss: %.3f' %(scores[1]*100,scores[0]))

NameError: name 'model' is not defined

In [ ]:
pyplot.plot(history.history['accuracy'],label='train')
pyplot.plot(history.history['val_accuracy'],label='test')
pyplot.legend()
pyplot.show()